# Analyze Excel Comments

Brett Deaton - Oct 2021

In an Excel spreadsheet, the Learning-15 team records data on which sessions were offered to which groups when. In a comment in some cells we record how many participants attended. We want a quick way to analyze the participation data. Some questions:
* How many people participate from a given group?
* ... in a given topic?
* ... on average?

### Setup

Load the Excel spreadsheet and create dictionaries of the column headers and row names.

Note, some of the following operations might be simpler using the pandas libary. But because we need the openpyxl library (I presume) to access cell comments, we're going to avoid using other database libraries.

In [ ]:
from openpyxl import load_workbook

In [ ]:
# Open the spreadsheet
filename = "comments_test.xlsx"
wb = load_workbook(filename)
ws = wb["Scheduling"]

In [ ]:
# View session names
headers = { cell.column_letter : cell.value for cell in ws[1] } # build dict from first row
print("Session names by column letter:")
for key, value in headers.items():
    if key>"E": # skip printing columns A-E which hold info specific to each team
        print(f"  {key}  {value}")

In [ ]:
# View supervisors
supervisors = { cell.row : cell.value for cell in ws["A"] } # build dict from first column
del supervisors[1] # get rid of the header in the first row
print("Supervisors by row number:")
for key, value in supervisors.items():
    print(f"  {key:>2}  {value}")

### Extract Comments

The comments extracted from each cell have a messy format, including a bunch of metadata and warnings, like the following:

```
[Threaded comment]

Your version of Excel allows you to read this threaded comment; however, any edits to it will get removed if the file is opened in a newer version of Excel. Learn more: https://go.microsoft.com/fwlink/?linkid=870924

Comment:
    14
```

In that example, we only need the number `14` following `Comment:`. We'll extract that using regular expressions.

In [ ]:
import re # to extract the relevant data from the metadata polluting Excel comments

In the following, we read the comments into a dictionary, retaining only the tasty bits. (Here's a stackoverflow example demonstrating
[how to read Excel comments](https://stackoverflow.com/questions/56010234/is-it-possible-to-read-excel-comments-with-pandas) using openpyxl library.)
Each dict value is a comment, each dict key is a tuple representing the column letter and row number of that cell.

In [ ]:
# Compile a regular expression to extract the relevant number from the metadata
p = re.compile(r"Comment:\s*(\w.*)$")
# The above regular expression captures strings that obey the following rules:
# 1. `Comment:` follow this word
# 2. `\s*` and are preceded by multiple whitespaces
# 3. `(\w.*)` (the captured part is a string starting with a nonwhitespace character)
# 4. `$` and are anchored to the end of the input string

# This dict will hold key:value pairs that locate the comment like this
#   key:   a tuple specifying the (column, row) of the comment
#   value: the comment as a string
attendance = {}

for row in ws.rows:
    for cell in row:
        if cell.comment is not None:
            match = p.search(cell.comment.text)
            attendance[(cell.column_letter, cell.row)] = match.group(1)

print(attendance)

In [ ]:
# Pretty-print the number of attendees for each session recorded
print("SESSION          SUPERVISOR          ATTENDANCE")
for (col, row), num_present in attendance.items():
    print(f"{headers[col]:15.15}  {supervisors[row]:18.18}  {num_present}")

### To Do

Some improvements or fixes to make:
* Eliminate the messy metadata in the Excel comments *within  Excel* so you don't have to resort to regular expressions *within this script*.
* Calculate total sum of attendees for a given session.
* Calculate average attendees from a given team.
* Make this robust against non-numeric comments that might also be present in the Excel notebook.